## GridSearchCV

In [13]:
#필요한 패키지 import

from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Dropout

import tensorflow as tf
import IPython

#keras Tuner 설치

!pip install -U keras-tuner
import kerastuner as kt

In [ ]:
#데이터셋 불러오기

(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

#데이터 정규화

X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [10]:
#모델 제작
def model_builder(hp):
    model = keras.Sequential()
    model.add(Flatten(input_shape=(28,28))) #입력층

    hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
    model.add(Dense(units= hp_units, activation = 'relu'))
    model.add(Dense(10, activation = 'softmax'))

    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                  loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics = ['accuracy'])
    
    return model

#튜너 제작

tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy',
                     max_epochs =  10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

#callback 함수 지정

class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait=True)

INFO:tensorflow:Reloading Oracle from existing project my_dir/intro_to_kt/oracle.json


In [11]:
#하이퍼파라미터 탐색

tuner.search(X_train, y_train, epochs = 10, validation_data = (X_test, y_test), callbacks = [ClearTrainingOutput()])


Trial 31 Complete [00h 00m 43s]
val_accuracy: 0.8499000072479248

Best val_accuracy So Far: 0.8881999850273132
Total elapsed time: 00h 15m 40s
INFO:tensorflow:Oracle triggered exit


In [12]:
#모델 평가

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
하이퍼 파라미터 검색이 완료되었습니다. 
최적화된 첫 번째 Dense 노드 수는 {best_hps.get('units')} 입니다.
최적의 학습 속도는 {best_hps.get('learning_rate')} 입니다.
""")


하이퍼 파라미터 검색이 완료되었습니다. 
최적화된 첫 번째 Dense 노드 수는 320 입니다.
최적의 학습 속도는 0.001 입니다.



## RandomSearchCV

In [16]:
#모델 제작
def model_builder(hp):
    model = keras.Sequential()
    model.add(Flatten(input_shape=(28,28))) #입력층

    hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
    model.add(Dense(units= hp_units, activation = 'relu'))
    hp_dropout = hp.Float('dropout', 0, 0.5, step=0.1)
    Dropout(rate=hp_dropout)
    
    model.add(Dense(10, activation = 'softmax'))

    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                  loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics = ['accuracy'])
    
    return model

#튜너 제작

tuner = kt.RandomSearch(model_builder,
                     objective = 'val_accuracy',
                     max_trials =  10,
                     directory = 'my_dir_2',
                     project_name = 'intro_to_kt_2')

#callback 함수 지정

class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait=True)

In [18]:
#하이퍼파라미터 탐색

tuner.search(X_train, y_train, epochs = 10, validation_data = (X_test, y_test), callbacks = [ClearTrainingOutput()])

Trial 10 Complete [00h 01m 05s]
val_accuracy: 0.8824999928474426

Best val_accuracy So Far: 0.888700008392334
Total elapsed time: 00h 12m 31s
INFO:tensorflow:Oracle triggered exit


In [20]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
하이퍼 파라미터 검색이 완료되었습니다. 
최적화된 첫 번째 Dense 노드 수는 {best_hps.get('units')} 입니다.
최적의 학습 속도는 {best_hps.get('learning_rate')} 입니다.
최적의 드롭 아웃은 {best_hps.get('dropout')}
""")


하이퍼 파라미터 검색이 완료되었습니다. 
최적화된 첫 번째 Dense 노드 수는 320 입니다.
최적의 학습 속도는 0.001 입니다.
최적의 드롭 아웃은 0.1



In [39]:
#최고 성능으로 모델 다시 만들기

model = tuner.hypermodel.build(best_hps)

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 784)               0         
                                                                 
 dense_6 (Dense)             (None, 320)               251200    
                                                                 
 dense_7 (Dense)             (None, 10)                3210      
                                                                 
Total params: 254,410
Trainable params: 254,410
Non-trainable params: 0
_________________________________________________________________


In [41]:
#early stop 적용하지 않고 최적 하이퍼파라미터로 모델학습하기

model.fit(X_train, y_train, epochs = 10, validation_data = (X_test, y_test))

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


1875/1875 [==============================] - 9s 4ms/step - loss: 0.4826 - accuracy: 0.8289 - val_loss: 0.4098 - val_accuracy: 0.8486
Epoch 2/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3615 - accuracy: 0.8684 - val_loss: 0.3821 - val_accuracy: 0.8650
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3243 - accuracy: 0.8797 - val_loss: 0.3730 - val_accuracy: 0.8646
Epoch 4/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3013 - accuracy: 0.8887 - val_loss: 0.3379 - val_accuracy: 0.8779
Epoch 5/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2807 - accuracy: 0.8953 - val_loss: 0.3558 - val_accuracy: 0.8706
Epoch 6/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2659 - accuracy: 0.8995 - val_loss: 0.3469 - val_accuracy: 0.8772
Epoch 7/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2542 - accuracy: 0.9041 - val_loss: 0.3488 - val_accuracy: 0.8812
Epo

In [43]:
#학습 평가

model.evaluate(x=X_test, y=y_test, verbose=1)

313/313 [==============================] - 1s 2ms/step - loss: 0.3335 - accuracy: 0.8896


[0.3334753215312958, 0.8895999789237976]

In [44]:
#early stop 저장 경로

checkpoint_filepath = "FMbest.hdf5"

In [45]:
#early stop 설정1 : 변화 없을 시 최대 갱신 횟수

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

#early stop 설정1 : 저장 단위(에포크), 저장 값(가중치), best만 저장

save_best = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
    save_weights_only=True, mode='auto', save_freq='epoch', options=None)

In [46]:
#early stop 적용하여 모델 학습 진행하기

model.fit(X_train, y_train, epochs = 10, validation_data = (X_test, y_test), callbacks=[early_stop, save_best])

Epoch 1/10
1869/1875 [============================>.] - ETA: 0s - loss: 0.2159 - accuracy: 0.9184
Epoch 1: val_loss improved from inf to 0.33566, saving model to FMbest.hdf5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2159 - accuracy: 0.9184 - val_loss: 0.3357 - val_accuracy: 0.8866
Epoch 2/10
1874/1875 [============================>.] - ETA: 0s - loss: 0.2072 - accuracy: 0.9226
Epoch 2: val_loss did not improve from 0.33566
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2072 - accuracy: 0.9226 - val_loss: 0.3524 - val_accuracy: 0.8832
Epoch 3/10
1863/1875 [============================>.] - ETA: 0s - loss: 0.2003 - accuracy: 0.9244
Epoch 3: val_loss did not improve from 0.33566
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2001 - accuracy: 0.9244 - val_loss: 0.3682 - val_accuracy: 0.8781
Epoch 4/10
1865/1875 [============================>.] - ETA: 0s - loss: 0.1938 - accuracy: 0.9272
Epoch 4: val_loss did not improve from 

In [48]:
#callback으로 최고 성능 모델 부르기

model.load_weights(checkpoint_filepath)

In [49]:
#모델 예측하기

model.predict(X_test[0:1])

array([[1.0479223e-10, 1.0205348e-12, 4.8267952e-12, 1.1370999e-12,
        8.5509676e-11, 3.9878526e-07, 6.2837392e-12, 1.5809326e-03,
        4.9421972e-10, 9.9841869e-01]], dtype=float32)

In [50]:
#학습 평가

test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.3357 - accuracy: 0.8866 - 552ms/epoch - 2ms/step
